### imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime # to track the time each epoch takes
print('did it')

did it


# load dataset

In [2]:
train_dataset = torchvision.datasets.CIFAR10(
    root = '.',
    train = True,
    transform=transforms.ToTensor(),
    download= True
)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-d8107ceac1d1>", line 1, in <module>
    train_dataset = torchvision.datasets.CIFAR10(
  File "/usr/local/lib/python3.8/dist-packages/torchvision/datasets/cifar.py", line 66, in __init__
    self.download()
  File "/usr/local/lib/python3.8/dist-packages/torchvision/datasets/cifar.py", line 144, in download
    download_and_extract_archive(self.url, self.root, filename=self.filename, md5=self.tgz_md5)
  File "/usr/local/lib/python3.8/dist-packages/torchvision/datasets/utils.py", line 427, in download_and_extract_archive
    download_url(url, download_root, filename, md5)
  File "/usr/local/lib/python3.8/dist-packages/torchvision/datasets/utils.py", line 130, in download_url
    url = _get_redirect_url(url, max_hops=max_redirect_hops)
  File "/usr/local/lib/python3.8/dist-p

TypeError: object of type 'NoneType' has no len()

In [ ]:
# BEHAVES DIFFERENTLY FROM MNIST!!  
# IT RETURNS A NUMPY ARRAY
train_dataset.data

In [ ]:
train_dataset.data.max()  # meaning it is in 255 (bear in mind that it is only this attribute)

In [ ]:
train_dataset.data.shape # N,H,W,C

In [ ]:
# Instead of being stores in tensors, it is stored in a list
train_dataset.targets # vector of 0 to 9


In [ ]:
test_dataset = torchvision.datasets.CIFAR10(
    root = '.',
    train = False,
    transform=transforms.ToTensor(),
    download = True
)

In [ ]:
K = len(set(train_dataset.targets))
print(f'number of classes: {K}')

#### data loader

In [ ]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size = batch_size, shuffle=False)

In [ ]:
tmp_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size = 1, shuffle=True)

for x,y in tmp_loader:
    print(x)  # it is now a tensor!
    print(x.shape) # (1,3,32,32) --> N,C,H,W  Behind the scenes it puts the color channel before H and W
    print(y)
    break

### model definition

In [ ]:
class CNN(nn.Module):
    def __init__(self, K):
        super(CNN, self).__init__()   # call the init constructor of the nn.Module. This way, we are only adding attributes.
        
        # In this case, we only the layers that have parameters.
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=2)  # in_channels is the number of channels of the input image (or filter)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2) # out_channels is the number of fiters used in this layer
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2)
        
        self.fc1 = nn.Linear(128 * 3 * 3, 1024)
        self.fc2 = nn.Linear(1024, K)
        
        
    # instead of treating the relu as modules, we can treat them as functions. We can access them via torch funtional
    def forward(self, x):   # this is where we pass the input into the module
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)    
        #x.view(-1, 128*3*3) 
        x = F.dropout(x, p=0.5)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.2)
        x = self.fc2(x)
        return x
    

        
        

#### how to compute the nn.Linear(?, 512)??

Following the formula:

h1 = (32 + 2*0 -1 * (3 -1) -1 / 2) +1

h2 = (h3 + 2*0 -1 * (3 -1) -1 / 2) +1

h3 = (h2 + 2*0 -1 * (3 -1) -1 / 2) +1

? = h3 * h3 * # feature maps

In [ ]:
import math
h1 = math.floor(((32 + 2*0 -1 * (3 -1) -1) / 2) +1)
h2 = math.floor(((h1 + 2*0 -1 * (3 -1) -1) / 2) +1)
h3 = math.floor(((h2 + 2*0 -1 * (3 -1) -1) / 2) +1)
h3



In [ ]:
# instantiate the model
model = CNN(K)

#### move model to cuda

In [ ]:
print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # if cuda is available use CUDA
print(device)
model.to(device)  # this transfers all the parameters of the model to the GPU!!!!!!

#### loss and optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

#### batch gradient descent

In [ ]:
def batch_gd(model, criterion, optimizer, train_loader, test_loader, epochs):
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)
    
    for it in range(epochs):
        t0 = datetime.now()
        train_loss = []
        for inputs, targets in train_loader:
            # move data in gpu
            inputs, targets = inputs.to(device), targets.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward pass
            outputs = model(inputs) # inputs.shape = (N,H,W,1) 
            loss = criterion(outputs, targets)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
            
        train_loss = np.mean(train_loss)
        test_loss = []
        
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss.append(loss.item())
        test_loss = np.mean(test_loss)
        
        # save losses
        train_losses[it] = train_loss
        test_losses[it] = test_loss
        
        dt = datetime.now() - t0
        print(f'epoch {it+1}/{epochs}, train_loss: {train_loss:.4f}, test_loss: {test_loss:.4f}, duration: {dt}')
    
    return train_losses, test_losses
            
            

In [ ]:
train_losses, test_losses = batch_gd(model, criterion, optimizer, train_loader, test_loader, epochs=15)

In [ ]:
plt.plot(train_losses, label='train loss')
plt.plot(test_losses, label='test loss')
plt.legend()
plt.show()

#### Accuracy (train and test)

In [ ]:
# accuracy

n_correct = 0.
n_total = 0.

for inputs, targets in train_loader:
    inputs, targets = inputs.to(device), targets.to(device)
    
    outputs = model(inputs)
    
    # get prediction
    ## torch.max returns both max and argmax
    _, predictions = torch.max(outputs,1) # no need for the softmax!
    # .max(outputs, 1) 1 is for the axis = 1. That is why we that the max over the columns!!
    
    #print(predictions.shape) #128
    
    #update counts
    n_correct += (predictions == targets).sum().item()
    # why .item()?  --> since this is all in pytorch, to bring it back to python, we have to call .item()   That obj it has other thigs, such as grads and stuffds like that
    n_total += targets.shape[0]

train_accuracy = n_correct / n_total

n_correct = 0.
n_total = 0.
for inputs, targets in test_loader:
    inputs, targets = inputs.to(device), targets.to(device)
    outputs = model(inputs)
    
    # get prediction
    ## torch.max returns both max and argmax
    _, predictions = torch.max(outputs,1) # no need for the softmax!
    # .max(outputs, 1) 1 is for the axis = 1. That is why we that the max over the columns!!
    
    #print(predictions.shape) #128
    
    #update counts
    n_correct += (predictions == targets).sum().item()
    # why .item()?  --> since this is all in pytorch, to bring it back to python, we have to call .item()   That obj it has other thigs, such as grads and stuffds like that
    n_total += targets.shape[0]

test_accuracy = n_correct / n_total

print(f'train acc: {train_accuracy:.4f}, test acc: {test_accuracy:.4f}')

#### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
import itertools

def plot_confusion_matrix(cm, classes, normalize = False, title='confusion matrix', cmap = plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:,np.newaxis]
        print('normalized confusion matrix')
    else:
        print('confusion matrix without normalization')
        
    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation = 45)
    plt.yticks(tick_marks, classes)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j,i, format(cm[i,j], fmt),
                 horizontalalignment = 'center',
                 color ='white' if cm[i,j] > thresh else 'black')
    
    plt.tight_layout()
    plt.ylabel('true label')
    plt.xlabel('predicted label')
    plt.show()

In [ ]:
# get prediction in an array to compute confusion matrix

x_test = test_dataset.data
y_test = test_dataset.targets
p_test = np.array([])

for inputs, targets in test_loader:
    inputs, targets = inputs.to(device), targets.to(device)
    
    outputs = model(inputs)
    
    # get prediction
    ## torch.max returns both max and argmax
    _, predictions = torch.max(outputs,1) # no need for the softmax!
    
    p_test = np.concatenate((p_test, predictions.cpu().numpy()))  # this only works because test_loader is not random!!!!!

cm = confusion_matrix(y_test, p_test)
plot_confusion_matrix(cm, list(range(10)))

In [ ]:
labels = '''airplane
automobile
bird
cat
deer
dog
frog
horse
ship
truck'''.split('\n')

In [ ]:
labels

In [ ]:
p_test = p_test.astype(np.uint8)
misclassified_idxs = np.where(p_test!=y_test)[0]
i = np.random.choice(misclassified_idxs) # select one of the miclassifed idxs
plt.imshow(x_test[i].reshape(32,32,3))
plt.title('true label: %s predicted %s' % (labels[y_test[i]], labels[p_test[i]]))
plt.show()